In [3]:
from google.colab import drive
drive.mount('/content/drive')

folder_path = "/content/drive/MyDrive/DSE4211/"

Mounted at /content/drive


In [4]:
import os
print(os.listdir("/content/drive/MyDrive/DSE4211/"))


['btc_price_data.csv', 'apple_price_data.csv', 'JNJ_price_data.csv', 'Xiaomi_price_data.csv', 'BNB_price_data.csv', 'USDT_price_data.csv', 'Gold(Tether)_price_data.csv', 'Xiaomi_price_data.gsheet']


In [6]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
file_paths = {
    "Apple": "/content/drive/MyDrive/DSE4211/apple_price_data.csv",
    "BNB": "/content/drive/MyDrive/DSE4211/BNB_price_data.csv",
    "Bitcoin": "/content/drive/MyDrive/DSE4211/btc_price_data.csv",
    "JNJ": "/content/drive/MyDrive/DSE4211/JNJ_price_data.csv",
    "USDT": "/content/drive/MyDrive/DSE4211/USDT_price_data.csv",  # Fixed at 5%, excluded from training
    "Xiaomi": "/content/drive/MyDrive/DSE4211/Xiaomi_price_data.csv",
    "Gold_Tether": "/content/drive/MyDrive/DSE4211/Gold(Tether)_price_data.csv",
}

# Read and preprocess data
dfs = {}
for asset, path in file_paths.items():
    df = pd.read_csv(path)
    df.index = pd.date_range(start="2023-01-01", periods=len(df), freq="D")
    if "Close" in df.columns:
        df = df[["Close"]].rename(columns={"Close": asset})
    dfs[asset] = df

merged_df = pd.concat(dfs.values(), axis=1)  # Merge assets
merged_df = merged_df.iloc[2:].apply(pd.to_numeric, errors='coerce')
merged_df.fillna(method='ffill', inplace=True)  # Fill missing values

print("Cleaned Data (First 5 Rows):")
print(merged_df.head())

# Compute daily returns
returns = merged_df.pct_change().dropna()
returns_ex_usdt = returns.drop(columns=["USDT"], errors="ignore")  # Exclude USDT from training

# Prepare features and targets
X = returns_ex_usdt.shift(1).dropna()
y = returns_ex_usdt.loc[X.index]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest for each asset (excluding USDT)
rf_models = {}
predicted_returns = {}

for asset in returns_ex_usdt.columns:
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train_scaled, y_train[asset])
    rf_models[asset] = rf
    predicted_returns[asset] = rf.predict(X_test_scaled)[-1]

# Convert predictions into DataFrame
predicted_returns_df = pd.DataFrame(predicted_returns, index=["Predicted Return"])
print("\nPredicted Returns:")
print(predicted_returns_df)

# Ensure no negative weights and normalize to 95%
predicted_returns_arr = np.maximum(0, np.array(list(predicted_returns.values())))
allocation_weights = predicted_returns_arr / np.sum(predicted_returns_arr) * 0.95

# Create allocation DataFrame and add USDT fixed at 5%
allocation_df = pd.DataFrame({"Asset": returns_ex_usdt.columns, "Optimal Allocation (%)": allocation_weights * 100})
usdt_allocation = pd.DataFrame({"Asset": ["USDT"], "Optimal Allocation (%)": [5.0]})
allocation_df = pd.concat([allocation_df, usdt_allocation], ignore_index=True)

print("\nOptimal Portfolio Allocation:")
print(allocation_df)


<ipython-input-6-63417843d63d>:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.fillna(method='ffill', inplace=True)  # Fill missing values


Cleaned Data (First 5 Rows):
                 Apple         BNB       Bitcoin         JNJ      USDT  \
2023-01-03  123.632515  244.136978  16625.080078  166.303696  0.999692   
2023-01-04  124.907715  245.535904  16688.470703  168.114288  0.999771   
2023-01-05  123.583092  246.133362  16679.857422  166.873016  0.999759   
2023-01-06  128.130234  259.119690  16863.238281  168.226303  0.999763   
2023-01-07  128.654144  256.422852  16836.736328  163.867798  0.999697   

            Xiaomi  Gold_Tether  
2023-01-03   11.22  1817.210083  
2023-01-04   11.50  1817.010986  
2023-01-05   11.62  1833.518555  
2023-01-06   11.24  1850.594238  
2023-01-07   12.10  1830.700317  

Predicted Returns:
                     Apple       BNB   Bitcoin       JNJ    Xiaomi  \
Predicted Return  0.002312  0.014772  0.011868  0.000181 -0.000055   

                  Gold_Tether  
Predicted Return     -0.00351  

Optimal Portfolio Allocation:
         Asset  Optimal Allocation (%)
0        Apple             